# Unitary optimiser

This notebook contains code to take a unitary and specific interferommeter and finds the phase list of the closest unitary that the interferometer can reach.

In [ ]:
%load_ext autoreload
%autoreload 2

In [22]:
import numpy as np
import strawberryfields as sf
import random
from scipy.linalg import block_diag
from scipy.optimize import minimize

# Construct interferometer

First construct interferometer

In [34]:

def clements_4_mode(component_params):
    ''' 
    args:
        * component_params 
            - list of all parameters for a 4 mode clements scheme
            - component_params[0:4] - Phases
            - component_params[4:21] - BS parameters in pairs

    returns:
        * U - compiled unitary implemented

    '''
    phases = component_params[0:4]
    BS_params = component_params[4:]
    
    Uphase = np.diag([np.exp(phases[0]*1j),np.exp(phases[1]*1j),np.exp(phases[2]*1j),np.exp(phases[3]*1j)])
    BSargs = [
        (BS_params[0], BS_params[1]),
        (BS_params[2], BS_params[3]),
        (BS_params[4], BS_params[5]),
        (BS_params[6], BS_params[7]),
        (BS_params[8], BS_params[9]),
        (BS_params[10], BS_params[11])
    ]
    
    BSunitaries = [np.array([[np.exp(p*1j)*np.sin(q/2), np.cos(q/2)], [np.exp(p*1j)*np.cos(q/2), -np.sin(q/2)]]) for q,p in BSargs] #Universal arrangment

    UBS1 = block_diag(*BSunitaries[0:2])
    UBS2 = block_diag([[1]], BSunitaries[2], [[1]])
    UBS3 = block_diag(*BSunitaries[3:5])
    UBS4 = block_diag([[1]], BSunitaries[5], [[1]])


    return Uphase @ UBS4 @ UBS3 @ UBS2 @ UBS1




# Define optimsier functions

In [ ]:
def cost_func(test_params,targe_U):


    W = clements_4_mode(test_params)
    U = targe_U


    return np.sum(abs(W-U)**2)

def fidelity(U_opt,U):
    return np.real(np.matrix.trace(U_opt.conj().T @ U)/4)

# Test optimiser

In [35]:
target_U = sf.utils.random_interferometer(4,real=False)

options = {'maxiter':250,
          'xatol':0.0001,
          'fatol':0.01}
method = 'Nelder-Mead'


param_guess =[2*np.pi*random.uniform(0,1) for _ in range(22)]

res = minimize(cost_func,x0=param_guess,args = (target_U),method='BFGS')

U_opt = clements_4_mode(res.x)

print(f'Fidelity = {fidelity(target_U,U_opt)}')

Fidelity = 0.9999999999804624


In [ ]:
def interferometer(params):

    '''Args
    params: dict corresponding to values for symbols
            '''
    U = np.eye(4):

    U[0,0] = 0.25*(1-np.exp(1j*params['MZIa1'])*np.exp(1j*params['Pha5'] + 1j*params['Pha1']))*(1 - np.exp(1j*params['MZIa3']))